In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
from windrose import WindroseAxes
from datetime import datetime, timedelta
import nbformat
nb = nbformat.v4.new_notebook()

In [58]:
df = pd.read_csv('../data/benin-malanville.csv')

In [15]:
start_time = datetime(2025, 5, 20, 9, 53, 0)
df['Timestamp'] = [start_time + timedelta(minutes=i) for i in range(len(df))]

In [23]:
numeric_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'WDstdev', 'WD', 'BP', 'Precipitation', 'TModA', 'TModB']
print("\nSummary Statistics:")
print(df[numeric_cols].describe())


Summary Statistics:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.100000       0.000000   
25%         

In [25]:
print("\nMissing Values:")
missing = df.isna().sum()
print(missing)
print("\nColumns with >5% missing values:")
missing_percent = (missing / len(df)) * 100
print(missing_percent[missing_percent > 5])


Missing Values:
Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64

Columns with >5% missing values:
Comments    100.0
dtype: float64


In [27]:
z_scores = df[['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']].apply(lambda x: np.abs(stats.zscore(x)))
outliers = (z_scores > 3).any(axis=1)
print("\nNumber of rows with outliers (|Z| > 3):", outliers.sum())


Number of rows with outliers (|Z| > 3): 7740


In [29]:
key_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
for col in key_cols:
    if df[col].isna().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

In [37]:
df.to_csv('../data/benin_clean.csv', index=False)


In [39]:
plt.figure(figsize=(12, 6))
plt.plot(df['Timestamp'], df['GHI'], label='GHI')
plt.plot(df['Timestamp'], df['DNI'], label='DNI')
plt.plot(df['Timestamp'], df['DHI'], label='DHI')
plt.plot(df['Timestamp'], df['Tamb'], label='Tamb')
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('Time Series: GHI, DNI, DHI, and Tamb')
plt.legend()
plt.grid(True)
plt.savefig('time_series.png')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3628\3986069225.py:11: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('time_series.png')
C:\Users\Administrator\anaconda3\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


In [41]:
cleaning_impact = df.groupby('Cleaning')[['ModA', 'ModB']].mean()
print("\nAverage ModA and ModB by Cleaning Flag:")
print(cleaning_impact)


Average ModA and ModB by Cleaning Flag:
                ModA        ModB
Cleaning                        
0         236.524253  228.816071
1         307.229278  301.972165


In [43]:
cleaning_impact.plot(kind='bar', figsize=(8, 6))
plt.title('Average ModA and ModB Pre/Post Cleaning')
plt.xlabel('Cleaning Flag')
plt.ylabel('Average Value')
plt.savefig('cleaning_impact.png')

In [45]:
corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
corr_matrix = df[corr_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.savefig('correlation_heatmap.png')

In [47]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.scatter(df['WS'], df['GHI'], alpha=0.5)
plt.xlabel('WS')
plt.ylabel('GHI')
plt.title('WS vs. GHI')

plt.subplot(1, 3, 2)
plt.scatter(df['WSgust'], df['GHI'], alpha=0.5)
plt.xlabel('WSgust')
plt.ylabel('GHI')
plt.title('WSgust vs. GHI')

plt.subplot(1, 3, 3)
plt.scatter(df['WD'], df['GHI'], alpha=0.5)
plt.xlabel('WD')
plt.ylabel('GHI')
plt.title('WD vs. GHI')
plt.tight_layout()
plt.savefig('scatter_ws_ghi.png')

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.scatter(df['RH'], df['Tamb'], alpha=0.5)
plt.xlabel('RH')
plt.ylabel('Tamb')
plt.title('RH vs. Tamb')

plt.subplot(1, 2, 2)
plt.scatter(df['RH'], df['GHI'], alpha=0.5)
plt.xlabel('RH')
plt.ylabel('GHI')
plt.title('RH vs. GHI')
plt.tight_layout()
plt.savefig('scatter_rh.png')

In [49]:
fig = plt.figure(figsize=(8, 8))
ax = WindroseAxes.from_ax(fig=fig)
ax.bar(df['WD'], df['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title('Wind Rose: Wind Speed and Direction')
plt.savefig('wind_rose.png')

In [51]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(df['GHI'], bins=20, edgecolor='black')
plt.xlabel('GHI')
plt.ylabel('Frequency')
plt.title('Histogram of GHI')

plt.subplot(1, 2, 2)
plt.hist(df['WS'], bins=20, edgecolor='black')
plt.xlabel('WS')
plt.ylabel('Frequency')
plt.title('Histogram of WS')
plt.tight_layout()
plt.savefig('histograms.png')

In [ ]:
fig = px.scatter(df, x='Tamb', y='GHI', size='RH', hover_data=['Timestamp'],
                 title='Bubble Chart: GHI vs. Tamb (Bubble Size = RH)')
fig.write_image('bubble_chart.png')